In [ ]:
import snowflake.snowpark as snowpark

from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
DB_NAME = "SUMMIT_25_AI_OBS_DEMO"
SCHEMA_NAME = "DATA"
STAGE_NAME = "DOCS"
WH_NAME = "COMPUTE_WH"

In [ ]:
-- List files in the stage to identify PDFs
LS @{{DB_NAME}}.{{SCHEMA_NAME}}.{{STAGE_NAME}}

## Step 1: Parse and Chunk Text from PDFs
We begin by parsing the content of uploaded PDFs and chunking the text using Snowflake's [PARSED_TEXT](https://docs.snowflake.com/sql-reference/functions/parse_document-snowflake-cortex) and [SPLIT_TEXT_RECURSIVE_CHARACTER](https://docs.snowflake.com/sql-reference/functions/split_text_recursive_character-snowflake-cortex) features. These steps structure the text into manageable segments optimized for retrieval. To ensure that the PDF parsing and chunking have been processed correctly, we run queries on the parsed and chunked tables. This step helps verify the integrity of the content.

Objective: **Transform unstructured content into indexed chunks for efficient search and retrieval.**

Key Outputs:
- SKO.HOP.PARSED_TEXT: Table containing the raw text.
- SKO.HOP.CORTEX_CHUNK: Chunked, searchable content.

In [1]:
-- Create a table to hold the extracted text from the PDF files loaded in the SKO_SKORAGHOP_LIVE_PROD.HOP.RAG stage

-- Complete the missing code (???) to use create a table called PARSED_TEXT

CREATE OR REPLACE TABLE {{DB_NAME}}.{{SCHEMA_NAME}}.PARSED_TEXT (relative_path VARCHAR(500), raw_text VARIANT);

SyntaxError: invalid syntax (579301450.py, line 1)

In [ ]:
INSERT INTO SUMMIT_25_AI_OBS_DEMO.DATA.PARSED_TEXT (relative_path, raw_text)
WITH pdf_files AS (
    SELECT DISTINCT
        METADATA$FILENAME AS relative_path
    FROM @SUMMIT_25_AI_OBS_DEMO.DATA.DOCS
    WHERE METADATA$FILENAME ILIKE '%.pdf'
      -- Exclude files that have already been parsed
      AND METADATA$FILENAME NOT IN (SELECT relative_path FROM PARSED_TEXT)
)
SELECT 
    relative_path,
    SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
        '@SUMMIT_25_AI_OBS_DEMO.DATA.DOCS',  -- Your stage name
        relative_path,  -- File path
        {'mode': 'layout'}  -- Adjust mode as needed ('layout', 'ocr')
    ) AS raw_text
FROM pdf_files;

In [ ]:
-- check the RAW_TEXT to ensure the PDF was parsed as expected
-- Complete the missing code (???) to check the RAW_TEXT to ensure the PDF was parsed as expected

SELECT *, SNOWFLAKE.CORTEX.COUNT_TOKENS('mistral-7b', RAW_TEXT) as token_count
FROM {{DB_NAME}}.{{SCHEMA_NAME}}.PARSED_TEXT;

In [ ]:
-- Use Snowflake's new SPLIT_TEXT_RECURSIVE_CHARACTER feature to chunk parsed text from the PDFs loaded in @SKO_SKORAGHOP_LIVE_PROD.HOP.RAG stage
-- Cortex SPLIT_TEXT_RECURSIVE_CHARACTER documentation link is https://docs.snowflake.com/sql-reference/functions/split_text_recursive_character-snowflake-cortex

-- Complete the missing code (???) to:
---- Create a new table called CORTEX_CHUNK to hold the chunked text from your PDF documents
---- Use Cortex SPLIT_TEXT_RECURSIVE_CHARACTER feature with a 2000 chunk size and 100 overlap size

CREATE OR REPLACE TABLE {{DB_NAME}}.{{SCHEMA_NAME}}.DOC_CHUNKS AS
WITH text_chunks AS (
    SELECT
        relative_path,
        SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(
            raw_text:content::STRING,  -- Extract the 'content' field from the JSON
            'markdown', -- Adjust to 'markdown' if needed
            2000,       -- Adjust chunk size
            100,        -- Adjust overlap size
            ['\n\n', '\n']    -- Adjust separators
        ) AS chunks
    FROM {{DB_NAME}}.{{SCHEMA_NAME}}.PARSED_TEXT
)
SELECT
    relative_path,
    c.value AS chunk  -- Extract each chunk of the parsed text
FROM text_chunks,
LATERAL FLATTEN(INPUT => chunks) c;

In [ ]:
-- check the CORTEX_CHUNK to ensure the PDF was chunked as expected
-- Complete the missing code (???) to check the CORTEX_CHUNK to ensure the PDF was chunked as expected for the PDF called "RAGWithoutAugmentation.pdf"

SELECT *, SNOWFLAKE.CORTEX.COUNT_TOKENS('mistral-7b', CHUNK) as token_count
FROM {{DB_NAME}}.{{SCHEMA_NAME}}.DOC_CHUNKS 
WHERE RELATIVE_PATH = 'Cortex_TSI.pdf'

In [ ]:
ALTER TABLE DOC_CHUNKS ADD COLUMN IF NOT EXISTS CHUNK_TOPIC VARCHAR(100);

UPDATE DOC_CHUNKS 
    SET CHUNK_TOPIC = SNOWFLAKE.CORTEX.COMPLETE('llama4-maverick', 
    concat('Categorize the following text as one of the following categories 
    [Customer Reference, Code Example, Benchmark, Technical Blog] 
    and only return the name of the category. No additional text.', CHUNK));

SELECT * FROM DOC_CHUNKS;    

In [ ]:
SELECT * FROM DOC_CHUNKS

## Step 2: Create Cortex Search Service
Next, we create a [Cortex Search Service](https://docs.snowflake.com/LIMITEDACCESS/cortex-search/cortex-search-overview#overview) that enables retrieval of relevant text chunks for any query. This service uses the CHUNK column from the chunked table as the indexed content.

Purpose: **Index and search chunked content to support the RAG pipeline.**

Command:
```sql
CREATE OR REPLACE CORTEX SEARCH SERVICE SKO.HOP.RAG_SEARCH_SERVICE ON SEARCH_COL WAREHOUSE = COMPUTE_WH TARGET_LAG = '1 day' AS SELECT  ...;
```

In [ ]:
SELECT 
        ('DOCUMENT_TITLE: ' || RELATIVE_PATH || 
        '\nDOCUMENT_TYPE: ' || CHUNK_TOPIC || 
        '\nDOCUMENT_TEXT:\n' || CHUNK) AS SEARCH_COL
        FROM {{DB_NAME}}.{{SCHEMA_NAME}}.DOC_CHUNKS;

In [ ]:
-- Create a search service over your new chunked pdf table that has one searchable text

CREATE OR REPLACE CORTEX SEARCH SERVICE {{DB_NAME}}.{{SCHEMA_NAME}}.SNOWFLAKE_BLOG_RETRIEVAL
    ON SEARCH_COL
    ATTRIBUTES CHUNK_TOPIC
    WAREHOUSE = COMPUTE_WH
    TARGET_LAG = '365 days'
    AS SELECT 
        RELATIVE_PATH,
        CHUNK_TOPIC,
        ('DOCUMENT_TITLE: ' || RELATIVE_PATH || 
        '\nDOCUMENT_TYPE: ' || CHUNK_TOPIC || 
        '\nDOCUMENT_TEXT:\n' || CHUNK) AS SEARCH_COL
    FROM {{DB_NAME}}.{{SCHEMA_NAME}}.DOC_CHUNKS;

## Step 3: Test Search Results with Experimental Configurations
We will now evaluate [Snowflake Cortex Experimental Knobs](https://docs.google.com/document/d/1HkHtDiY3CmzpSewCe_s9fpMNE5spOUvNSwr6CxFerqE/edit?usp=sharing) to fine-tune the retrieval service and analyze confidence scores and result rankings across configurations. These tests focus on boosting, recency, headers, and reranking to optimize search relevance.

**Configurations Tested:**
- **Boosted vs. Unboosted:** Compare the impact of keyword emphasis on rankings and scores.
- **Time-Based Decays:** Test how prioritizing recent documents affects relevance.
- **Header Boosts:** Evaluate the influence of structured headers (e.g., Markdown) on ranking.
- **Reranked vs. Non-Reranked:** Analyze trade-offs between query latency and search quality.

**Key Metrics:**
- **Confidence Scores:** Global relevance scores (0–3) for each result.
- **Result Rankings:** Position changes reveal the effectiveness of configurations.

By testing these configurations, we aim to enhance Cortex Search Service performance for specific use cases.

In [ ]:
# Define image in a stage and read the file
image=session.file.get_stream("@SKO_SKORAGHOP_LIVE_PROD.HOP.RAG/CortexSearchEnhancements.jpg", decompress=False).read() 
st.image(image, width=800)

In [ ]:
-- This query compares Cortex Search Service results across multiple experimental settings: 
---- boosted (using softBoosts), header boosted, and unboosted (default settings).

-- The results are presented side by side to analyze the impact of each configuration on confidence scores and document ranking for matching search columns.
-- This analysis helps evaluate the effectiveness of boosting and decay strategies in improving search relevance and recency-based ranking.

-- Missing code (???) has been completed to:
---- Call the SKO_SKORAGHOP_LIVE_PROD.HOP.RAG_SEARCH_SERVICE to test experimental configurations.
---- Use the query: "How can I augment my LLM prompts with relevant context in Snowpark?"
---- For the boosted_results section, apply softBoosts using the phrases "Augment" and "RAG."
---- Enable returnConfidenceScores to true for all configurations.

WITH boosted_results AS (
    SELECT DISTINCT
        VALUE:"SEARCH_COL"::STRING AS SearchColumn,
        VALUE:"@CONFIDENCE_SCORE"::STRING AS ConfidenceScore
    FROM (
        SELECT PARSE_JSON(
            SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
                'SKO_SKORAGHOP_LIVE_PROD.HOP.RAG_SEARCH_SERVICE',
                '{
                    "query": "How can I augment my llm prompts with relevant context in snowpark?",
                    "limit": 3,
                    "experimental": {
                        "softBoosts": [
                            { "phrase": "Augment" },
                            { "phrase": "RAG" }
                        ],
                        "reranker": "none",
                        "returnConfidenceScores": true
                    }
                }'
            )
        ) AS boosted_json
    ),
    LATERAL FLATTEN(input => boosted_json:"results")
),
header_boosted_results AS (
    SELECT DISTINCT
        VALUE:"SEARCH_COL"::STRING AS SearchColumn,
        VALUE:"@CONFIDENCE_SCORE"::STRING AS ConfidenceScore
    FROM (
        SELECT PARSE_JSON(
            SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
                'SKO_SKORAGHOP_LIVE_PROD.HOP.RAG_SEARCH_SERVICE',
                '{
                    "query": "How can I augment my llm prompts with relevant context in snowpark?",
                    "limit": 3,
                    "experimental": {
                        "headerBoost": {
                            "multiplier": 2,
                            "skipStopWords": true
                        },
                        "reranker": "none",
                        "returnConfidenceScores": true
                    }
                }'
            )
        ) AS header_boosted_json
    ),
    LATERAL FLATTEN(input => header_boosted_json:"results")
),
unboosted_results AS (
    SELECT DISTINCT
        VALUE:"SEARCH_COL"::STRING AS SearchColumn,
        VALUE:"@CONFIDENCE_SCORE"::STRING AS ConfidenceScore
    FROM (
        SELECT PARSE_JSON(
            SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
                'SKO_SKORAGHOP_LIVE_PROD.HOP.RAG_SEARCH_SERVICE',
                '{
                    "query": "How can I augment my llm prompts with relevant context in snowpark?",
                    "limit": 3,
                    "experimental": {
                        "returnConfidenceScores": true
                    }
                }'
            )
        ) AS unboosted_json
    ),
    LATERAL FLATTEN(input => unboosted_json:"results")
)
SELECT 
    COALESCE(b.SearchColumn, hb.SearchColumn, u.SearchColumn) AS SearchColumn,
    b.ConfidenceScore AS BoostedConfidenceScore,
    hb.ConfidenceScore AS HeaderBoostedConfidenceScore,
    u.ConfidenceScore AS UnboostedConfidenceScore
FROM
    boosted_results b
FULL OUTER JOIN header_boosted_results hb
    ON b.SearchColumn = hb.SearchColumn
FULL OUTER JOIN unboosted_results u
    ON COALESCE(b.SearchColumn, hb.SearchColumn) = u.SearchColumn
ORDER BY 
    CASE WHEN BoostedConfidenceScore IS NULL THEN 1 ELSE 0 END, 
    BoostedConfidenceScore DESC;

## Step 4: Pass Retrieved Content to LLMs
This step demonstrates how to pass retrieved contextual content to various LLMs using the Snowflake Cortex [`COMPLETE`](https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex) function. The process includes:

- **Retrieving Contextual Information**: Context is fetched from the search service.
- **Generating Structured Prompts**: The retrieved context is injected into prompts for LLMs.
- **LLM Interaction**: Prompts are passed to models like `mistral-7b`, `mistral-large2`, and `Anthropic Claude 3.5` for response generation.
- **Comparative Analysis**: Model outputs are compared for quality, relevance, and coherence.

Example Query:
```sql
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'claude-3-5-sonnet',
    CONCAT('Your context: ', (SELECT LISTAGG(CHUNK, ' ') FROM searchresults))
) AS RESPONSE
FROM searchresults;
```

**Queries to test the capabilities of the LLMs based on the PDF content:**
- What is the difference between semantic and lexical searches? Does a hybrid system exist?
- How can we optimize context retrieval in retrievel agumented geneartion for an LLM system?"'
- Can I use SQL in Snowflake to retrieve relevant context for my GPT prompt?
- What service runs fuzzy-search to retrieve context in Snowflake?

In [ ]:
# Query your Snowflake Cortex Search Service using the Snowpark Python API to retrieve and process search results.

# Complete the missing code (???) to:
## Specify your database 'SKO_SKORAGHOP_LIVE_PROD', your schema 'HOP', and your Cortex Search Service named 'RAG_SEARCH_SERVICE'
## Specify your SEARCH_COL as the column of interest

from snowflake.snowpark import Session
from snowflake.core import Root
root = Root(session)

transcript_search_service = (root
  .databases[DB_NAME]
  .schemas[SCHEMA_NAME]
  .cortex_search_services['SNOWFLAKE_BLOG_RETRIEVEL']
)

resp = transcript_search_service.search(
  query="""How does Snowflake simplify the deployment of retrieval-augmented generation (RAG) workflows?""",
  columns=['SEARCH_COL'],
  limit=3
)
results = resp.results

context_str = ""
for i, r in enumerate(results):
    context_str += f"Context document {i+1}: {r['SEARCH_COL']}\n****************\n"

print(context_str)
df = session.create_dataframe(resp.results)
df.create_or_replace_temp_view("searchresults")

In [ ]:
# Define the retriever class for interacting with the Cortex Search Service

# Complete the missing code (???) to:
## Specify your database 'SKO_SKORAGHOP_LIVE_PROD', your schema 'HOP', and your Cortex Search Service named 'RAG_SEARCH_SERVICE'
## Specify your SEARCH_COL as the column of interest
## Intialize retriever with your CortexSearchRetriever class
## Use "What are some components of the Snowflake Cortex offering? How do they work?" for the test_query

from typing import List
from snowflake.snowpark import Session
from snowflake.core import Root

# CortexSearchRetriever
class CortexSearchRetriever:
    def __init__(self, session: Session, limit_to_retrieve: int = 4):
        self._session = session
        self._limit_to_retrieve = limit_to_retrieve
        

    def retrieve(self, query: str) -> List[str]:
        root = Root(session)
        cortex_search_service = (
            root
            .databases["SKO_SKORAGHOP_LIVE_PROD"]
            .schemas["HOP"]
            .cortex_search_services["RAG_SEARCH_SERVICE"]
        )
        resp = cortex_search_service.search(
            query=query,
            columns=["SEARCH_COL"],
            limit=self._limit_to_retrieve,
        )
        return [row["SEARCH_COL"] for row in resp.results] if resp.results else []

# Initialize the retriever
retriever = CortexSearchRetriever(session=session, limit_to_retrieve=3)
test_query = "What are some components of the Snowflake Cortex offering? How do they work?"
retrieved_context = retriever.retrieve(query=test_query)
print(retrieved_context)